# レッスン 06 LangChain

LangChainは、2022年末に大規模言語モデル技術が爆発的に普及して以来、初めての本格的な大規模言語モデル開発フレームワークと言えるでしょう。大規模言語モデルは本質的に実際の問題を直接解決することはできず、単にテキストを分析、推論、生成できるブラックボックスに過ぎません。今日に至るまで、すべての開発者は大規模言語モデルの強力な能力を実際のアプリケーションシナリオと組み合わせる方法を模索し続けています。そのような中、LangChainの登場により、大規模言語モデルの開発が一気に簡単になりました。LangChainは、大規模言語モデル開発プロセスでよく使用される機能、ツール、フローなどをすべてコンポーネントとしてカプセル化し、開発者がレゴブロックを組み立てるように、さまざまなシナリオのニーズに適した大規模言語モデルアプリケーションを迅速に構築できるようにしました。

LangChainの最初のバージョンは2022年10月にオープンソース化され、現在でも非常に速いペースで反復的にアップグレードされ続けています。オープンソースのPython/TSフレームワークから徐々に発展し、「チェーン」や「エージェント」などのコアコンポーネントを形成し、現在ではエンタープライズレベルの段階に進み、LangChain AIへと発展しました。これは現在、エージェント技術分野で最大のオープンソースエコシステムを持ち、複数のオープンソースプロジェクトフレームワークを派生させ、それぞれが大規模言語モデルの技術分野において異なる開発タスクの役割を担っています。

比喩的に言えば、LangChainの機能的な位置づけは、実際には現在私たちが呼んでいるエージェント開発フレームワークではなく、大規模言語モデルの機能拡張ツールです。LangChainを活用すれば、GPT-3モデルの時代であっても、モデルに対話、記憶の保持、さらには構造化された出力などの機能を実現させることができます。

## LangChainインストール

`LangChain`を使用して大規模言語モデルアプリケーションを開発する場合、`LangChain`の依存パッケージをインストールする必要があります。インストールコマンドは以下の通りです:

LangChain >= 1.0.0、Python >= 3.11であることを確認してください。バージョンが低い場合は、`pip install --upgrade <パッケージ名>`を使用してアップグレードしてください。

In [ ]:
!pip install langchain

In [18]:
import langchain

print(langchain.__version__)

1.1.0


## Langchain は API を介してオンラインLLMに接続します

`LangChain`の開発を始める前に、まず呼び出し可能な大規模言語モデルを準備する必要があります。ここでは`DeepSeek`の大規模言語モデルを使用し、`DeepSeek`公式の`API_KEY`を使って呼び出しを行います(ここではすでにコード内に提供されています)。自分て使用する場合は、まず`DeepSeek`公式サイトで登録を行い、新しい`API_Key`を作成する必要があります。公式サイトのアドレスは以下の通りです: https://platform.deepseek.com/usage

まずは現在の実行環境においては`LangChain`を使用せず、`DeepSeek API`を直接使用してネットワーク接続性テストを行います。テストコードは以下の通りです:

In [ ]:
!pip install openai

In [ ]:
DeepSeek_API_KEY = "sk-ec46a9d5509b472484283cc83fe3a895"

In [17]:
from openai import OpenAI

client = OpenAI(api_key=DeepSeek_API_KEY, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "あなたは数学研究者です"},
        {"role": "user", "content": "ある檻の中に、鶏とウサギが合わせて35匹います。頭の数を数えたら35個、足の数を数えたら94本でした。鶏は2本足、ウサギは4本足です。鶏とウサギはそれぞれ何匹いるでしょうか?"},
    ],
)

print(response.choices[0].message.content)

これは典型的な「鶴亀算」の問題です。  
条件を整理すると：

- 鶏とウサギの総数：35匹
- 足の総数：94本
- 鶏の足：2本
- ウサギの足：4本

---

### 解法1：連立方程式で解く
鶏を \( x \) 匹、ウサギを \( y \) 匹とすると：

\[
\begin{cases}
x + y = 35 \\
2x + 4y = 94
\end{cases}
\]

第2式を2で割る：
\[
x + 2y = 47
\]

これと \( x + y = 35 \) を引くと：
\[
(x + 2y) - (x + y) = 47 - 35
\]
\[
y = 12
\]
\[
x = 35 - 12 = 23
\]

---

### 解法2：仮定法で解く
全部が鶏だと仮定すると、足の数は \( 35 \times 2 = 70 \) 本。  
実際は94本なので、差は \( 94 - 70 = 24 \) 本。  
ウサギは鶏より2本足が多いので、ウサギの数は \( 24 \div 2 = 12 \) 匹。  
鶏は \( 35 - 12 = 23 \) 匹。

---

**答え**：  
鶏 \( 23 \) 匹、ウサギ \( 12 \) 匹


`DeepSeek`モデルからの応答を正常に受信できれば、`DeepSeek API`が正常に使用でき、ネットワーク接続性も正常であることを示します。

次に考えるべきことは、このような`DeepSeek`公式`API`を`LangChain`にどのように組み込むかということです。実は非常に簡単で、`LangChain`内の`DeepSeek`コンポーネントを使用するだけで、上述のコードと同様に、同じ`DeepSeek API KEY`を使って直接大規模言語モデルと対話することができます。そのため、まず`LangChain`の`DeepSeek`コンポーネントをインストールする必要があります。インストールコマンドは以下の通りです:

In [ ]:
!pip install langchain-deepseek

`LangChain`に統合された`DeepSeek`モデルの依存パッケージをインストールした後、`init_chat_model`関数を使用して大規模言語モデルを初期化する必要があります。コードは以下の通りです:

In [ ]:
! pip install python-dotenv

In [15]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [16]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat")

question = "上智大学の理工学部の強さはどのくらいですか？"

result = model.invoke(question)
print(result.content)

上智大学の理工学部について、その強みや特徴を以下のようにご説明します。

### 総合的な評価と位置づけ

上智大学は日本を代表する難関私立大学（いわゆる「早慶上智」）の一角を占めており、高いブランド力と教育力を有しています。しかし、**理工学部に焦点を当てた場合、その評価は文系学部とは少し異なる視点が必要です。**

**全体的な評価：**
*   **国内での評価：** 歴史の長い旧帝大や早慶、東京工業大学、筑波大学などの大規模理工系学部と比較すると、設置が比較的新しい（2012年設置）こともあり、伝統や規模の面では一歩劣ると見なされることが多いです。
*   **強みの分野：** 上智の強みは「**国際性**」と「**少人数教育**」にあります。これは理工学部にも強く反映されています。

---

### 理工学部の具体的な強み

1.  **圧倒的な国際性と英語教育**
    *   多くの授業が英語で開講されており、留学生と共に学ぶ環境が整っています。
    *   留学制度が充実しており、世界中の提携大学（特に欧米の名門校）への留学機会が豊富です。
    *   研究者として世界で活躍するためには必須の英語力と国際感覚を、学生時代から磨くことができます。

2.  **少人数による手厚い教育**
    *   上智大学全体の特徴である「少人数教育」が理工学部でも徹底されています。
    *   教授と学生の距離が近く、研究室配属後は特に、きめ細かな指導を受けることができます。これはアカデミックな道を志す学生にとって大きなメリットです。

3.  **文系の強みを活かした「文理融合」の視点**
    *   上智大学は外国語学部や経済学部、法学部など文系学部が非常に強い総合大学です。
    *   そのため、他学部の科目を履修しやすく、技術だけでなく、社会や倫理、経済的な視点から物事を考える「**リベラル・アーツ**」的な素養を身につけることができます。これは、現代の複雑な社会課題を解決するエンジニアや研究者に求められる重要な能力です。

4.  **先進的な研究分野**
    *   グリーンケミストリー、環境エネルギー、生命科学、人工知能（AI）など、現代社会が直面する重要な課題に直結する研究に力を入れています。
    *   特に*

In [21]:
model = ChatDeepSeek(model="deepseek-reasoner")

result = model.invoke(question)
print(result.content)

上智大学の理工学部について、その「強さ」をいくつかの観点から説明します。

結論から言うと、**「特定の分野に強みを持つ、中規模でハイレベルな理工学部」** という位置付けです。旧帝大や東工大、早慶の理工学部ほどの広範な圧倒的知名度はありませんが、非常に個性的で質の高い教育・研究を行っています。

### 強みと特徴

1.  **国際性と少人数教育**
    *   上智大学全体の特徴である「国際性」が理工学部にも強く反映されています。多くの授業が英語で開講され、留学生の比率も高い環境です。このため、**「世界で活躍できる理工系人材」** を育てることに強みがあります。
    *   学生数が比較的少ないため、教授と学生の距離が近く、きめ細やかな指導を受けられます。アットホームな環境を求める学生には最適です。

2.  **特色ある学科編成**
    *   **物質生命理工学科**、**機能創造理工学科**、**情報理工学科**の3学科で構成され、伝統的な「機械」「電気」といったくくりではなく、学際的で現代的な分野に焦点を当てています。
    *   特に**情報理工学科**は、人工知能（AI）やデータサイエンスなど、現在の社会ニーズが非常に高い分野に力を入れており、入学難易度も高くなっています。

3.  **研究面での強み**
    *   大規模大学のような莫大な予算と人員はありませんが、**「グリーンケミストリー」** や **「ソフトマター物理学」**、**「ロボティクス」**、**「AI・機械学習」** などの分野で、世界的に評価されている研究があります。
    *   研究者一人あたりの論文の質や影響力は非常に高いという評価もあります。

4.  **立地とネットワーク**
    *   東京の四谷という都心にあるため、企業との共同研究やインターンシップの機会に恵まれています。
    *   上智大学のブランド力と国内外のネットワークは、就職活動においても強力な武器になります。

### 他の大学との比較と位置付け

*   **旧帝国大学（東大、京大、東北大など）・東京工業大学との比較**：
    *   研究の規模、予算、博士課程への進学実績、全体的な知名度ではこれらの大学に及びません。これらは日本を代表する「研究重点」の理工

In [22]:
result

AIMessage(content='上智大学の理工学部について、その「強さ」をいくつかの観点から説明します。\n\n結論から言うと、**「特定の分野に強みを持つ、中規模でハイレベルな理工学部」** という位置付けです。旧帝大や東工大、早慶の理工学部ほどの広範な圧倒的知名度はありませんが、非常に個性的で質の高い教育・研究を行っています。\n\n### 強みと特徴\n\n1.  **国際性と少人数教育**\n    *   上智大学全体の特徴である「国際性」が理工学部にも強く反映されています。多くの授業が英語で開講され、留学生の比率も高い環境です。このため、**「世界で活躍できる理工系人材」** を育てることに強みがあります。\n    *   学生数が比較的少ないため、教授と学生の距離が近く、きめ細やかな指導を受けられます。アットホームな環境を求める学生には最適です。\n\n2.  **特色ある学科編成**\n    *   **物質生命理工学科**、**機能創造理工学科**、**情報理工学科**の3学科で構成され、伝統的な「機械」「電気」といったくくりではなく、学際的で現代的な分野に焦点を当てています。\n    *   特に**情報理工学科**は、人工知能（AI）やデータサイエンスなど、現在の社会ニーズが非常に高い分野に力を入れており、入学難易度も高くなっています。\n\n3.  **研究面での強み**\n    *   大規模大学のような莫大な予算と人員はありませんが、**「グリーンケミストリー」** や **「ソフトマター物理学」**、**「ロボティクス」**、**「AI・機械学習」** などの分野で、世界的に評価されている研究があります。\n    *   研究者一人あたりの論文の質や影響力は非常に高いという評価もあります。\n\n4.  **立地とネットワーク**\n    *   東京の四谷という都心にあるため、企業との共同研究やインターンシップの機会に恵まれています。\n    *   上智大学のブランド力と国内外のネットワークは、就職活動においても強力な武器になります。\n\n### 他の大学との比較と位置付け\n\n*   **旧帝国大学（東大、京大、東北大など）・東京工業大学との比較**：\n    *   研究の規模、予算、博士課程への進学実

In [23]:
result.additional_kwargs

{'refusal': None,
 'reasoning_content': 'ふむ、上智大学の理工学部の評価について聞かれているね。ユーザーはおそらく進学先を検討している高校生か、あるいは他大学から編入を考えている学生だろう。学部の「強さ」を尋ねるということは、単なる偏差値ではなく、教育内容や研究実績といった実質的な部分を重視しているのかもしれない。\n\n上智は文系のイメージが強いから、理工学部の認知度が気になっているのかも。実際のところ、国際性や少人数教育が売りだけど、東工大や早慶理系と比べると規模で劣るのは事実だ。ユーザーがどこに重きを置いているかで答えが変わるな。\n\n研究面では特に地球環境工学科がユニークだし、他大学にはない特徴がある。でも就職先が伝統的に強いのは文系で、理工学部はまだ歴史が浅いからデータが少ないかも。そこは客観的事実と大学の特色をバランスよく伝える必要がある。\n\n英語教育や留学制度を求めるなら評価が高いけど、純粋な研究実績なら旧帝大の方が上だろう。ユーザーの優先順位を聞き返せるとベストだけど、この段階では中立に情報を整理してあげよう。'}

In [24]:
result.additional_kwargs["reasoning_content"]

'ふむ、上智大学の理工学部の評価について聞かれているね。ユーザーはおそらく進学先を検討している高校生か、あるいは他大学から編入を考えている学生だろう。学部の「強さ」を尋ねるということは、単なる偏差値ではなく、教育内容や研究実績といった実質的な部分を重視しているのかもしれない。\n\n上智は文系のイメージが強いから、理工学部の認知度が気になっているのかも。実際のところ、国際性や少人数教育が売りだけど、東工大や早慶理系と比べると規模で劣るのは事実だ。ユーザーがどこに重きを置いているかで答えが変わるな。\n\n研究面では特に地球環境工学科がユニークだし、他大学にはない特徴がある。でも就職先が伝統的に強いのは文系で、理工学部はまだ歴史が浅いからデータが少ないかも。そこは客観的事実と大学の特色をバランスよく伝える必要がある。\n\n英語教育や留学制度を求めるなら評価が高いけど、純粋な研究実績なら旧帝大の方が上だろう。ユーザーの優先順位を聞き返せるとベストだけど、この段階では中立に情報を整理してあげよう。'

ここで分かるように、わずか2行のコードで、`LangChain`内で`DeepSeek`モデルをスムーズに呼び出し、モデルの応答結果を得ることができます。`DeepSeek API`を使用する場合と比較して、`LangChain`を使用してモデルを呼び出す方が、間違いなくより簡単です。同時に、`DeepSeek`モデルだけでなく、`LangChain`は`OpenAI`、`Qwen`、`Gemini`など、他の多くの大規模言語モデルもサポートしています。`init_chat_model`関数内で異なるモデル名を指定するだけで、異なるモデルを呼び出すことができます。`LangChain`で開発する際に他の大規模言語モデルを使用したい場合は、まずモデルの`API_KEY`を取得し、次に対応するサードパーティの依存パッケージをインストールするだけで、同様に`init_chat_model`関数を使用してモデルを初期化し、`invoke`メソッドを呼び出してモデルの応答結果を得ることができます。`LangChain`がどの大規模言語モデルをサポートしているか、および各モデルに対応するサードパーティの依存パッケージについては、`LangChain`の公式ドキュメントで確認できます。アクセスリンクは以下の通りです：https://python.langchain.com/docs/integrations/chat/

![langchain-providers](https://github.com/yanwunhao/gonken-lesson-build-llm-from-scratch/blob/main/figs/langchain-providers.png?raw=true)